In [1]:
import pandas as pd
import polars as pl
import numpy as np
import requests
import json
pd.set_option('display.max_columns', None)


In [2]:
# movies      = pd.read_parquet("clean_datasets/movies.parquet")
# actors      = pd.read_parquet("clean_datasets/actors_movies.parquet")
# directors   = pd.read_parquet("clean_datasets/directors_movies.parquet")

In [3]:
# clean les genres et les spoken_languages production_companies	production_countries a divier en deux qui sont production_companies_name et production_companies_country

In [4]:
df = pd.read_csv("tmdb_update.csv")

df

col_to_drop = [
    "belongs_to_collection",
    "status_code",
    "status_message",
    "success",
]
df = df.drop(columns=col_to_drop)


In [5]:
import ast

col = [
    "genres",
    "spoken_languages",
    "production_companies",
    "production_countries"
]
for c in col:
    df[c] = df[c].apply(lambda x: ast.literal_eval(x))


df["genres"] = df["genres"].apply(lambda x: [i['name'] for i in x])
df["spoken_languages"] = df["spoken_languages"].apply(lambda x: [i['iso_639_1'] for i in x])
df["production_companies_name"] = df["production_companies"].apply(lambda x: [i['name'] for i in x])
df["production_companies_country"] = df["production_companies"].apply(lambda x: [i['origin_country'] for i in x])
# df["production_companies_country"] = df["production_companies"].apply(lambda x: [i['origin_country'] for i in x if i["origin_country"] != ""])
df["production_countries"] = df["production_countries"].apply(lambda x: [i['iso_3166_1'] for i in x])


In [6]:
df.drop(columns=["production_companies"], inplace=True)
df.columns

Index(['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'production_companies_name',
       'production_companies_country'],
      dtype='object')

In [7]:
tmdb_full = pd.read_csv("movies_datasets/tmdb_full.csv")

C:\Users\morga\AppData\Local\Temp\ipykernel_1272\55730879.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  tmdb_full = pd.read_csv("movies_datasets/tmdb_full.csv")


In [8]:
tmdb_full.head(4)

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
0,False,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,0,['Comedy'],NaN,3924,tt0029927,en,Blondie,Blondie and Dagwood are about to celebrate the...,2.852,/zBiHKhXklvTFwj4M1uEUcQGAVJ.jpg,['US'],1938-11-30,0,70,['en'],Released,The favorite comic strip of millions at last o...,Blondie,False,7.214,7,['Columbia Pictures'],['US']
1,False,NaN,0,['Adventure'],NaN,6124,tt0011436,de,Der Mann ohne Namen,NaN,1.091,/6xUbUCvndklbGVYiljHr34NTxSl.jpg,['DE'],1921-01-01,0,420,[],Released,NaN,"Peter Voss, Thief of Millions",False,0.000,0,[],[]
2,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.770,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,0,110,"['it', 'ja', 'pl', 'fr', 'de']",Released,The Intimate Secrets of Young Lovers,Love at Twenty,False,6.700,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']"
3,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,0,"['Drama', 'Comedy', 'Crime']",NaN,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,9.214,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,['FI'],1988-10-21,0,73,['fi'],Released,NaN,Ariel,False,7.046,248,['Villealfa Filmproductions'],['FI']


In [9]:
tmdb_full["production_companies_country"].fillna(value="[]", inplace=True)

In [10]:
tmdb_full
col = [
    "genres",
    "spoken_languages",
    "production_companies_name",
    "production_companies_country",
    "production_countries"
]
for c in col:
    tmdb_full[c] = tmdb_full[c].apply(lambda x: ast.literal_eval(x))

tmdb_full

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
0,False,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,0,[Comedy],NaN,3924,tt0029927,en,Blondie,Blondie and Dagwood are about to celebrate the...,2.852,/zBiHKhXklvTFwj4M1uEUcQGAVJ.jpg,[US],1938-11-30,0,70,[en],Released,The favorite comic strip of millions at last o...,Blondie,False,7.214,7,[Columbia Pictures],[US]
1,False,NaN,0,[Adventure],NaN,6124,tt0011436,de,Der Mann ohne Namen,NaN,1.091,/6xUbUCvndklbGVYiljHr34NTxSl.jpg,[DE],1921-01-01,0,420,[],Released,NaN,"Peter Voss, Thief of Millions",False,0.000,0,[],[]
2,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"[Drama, Romance]",NaN,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.770,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"[DE, FR, IT, JP, PL]",1962-06-22,0,110,"[it, ja, pl, fr, de]",Released,The Intimate Secrets of Young Lovers,Love at Twenty,False,6.700,41,"[Ulysse Productions, Unitec Films, Cinesecolo,...","[, NZ, IT, JP, DE, PL, ]"
3,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,0,"[Drama, Comedy, Crime]",NaN,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,9.214,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,[FI],1988-10-21,0,73,[fi],Released,NaN,Ariel,False,7.046,248,[Villealfa Filmproductions],[FI]
4,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,0,"[Drama, Comedy, Romance]",NaN,3,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",6.282,/nj01hspawPof0mJmlgfjuLyJuRN.jpg,[FI],1986-10-17,0,76,[en],Released,NaN,Shadows in Paradise,False,7.182,269,[Villealfa Filmproductions],[FI]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309567,False,/oGW8uF821i8VowzpmECbxk7QXNs.jpg,0,"[Crime, Drama]",NaN,957102,tt24962414,zh,重裝戰警,The film tells the story of the heavy-duty pol...,1.307,/tzF2BndyY3HIf2KjJR1FBYoIqTp.jpg,[CN],2022-04-01,0,71,[zh],Released,NaN,Hit Team,False,0.000,0,[],[]
309568,False,/oQ429AcD85ttxvOxAaYpETnAsW0.jpg,0,"[Action, War]",NaN,918692,tt17352384,ru,Гранит,Mozambique requests from Russia is being helpe...,1.238,/nqMk6TIUwqSMRc6QPKYguNr9Ql0.jpg,[UZ],2021-12-29,0,100,[ru],Released,NaN,Granite,False,5.000,3,[],[]
309569,False,/fJhruCWu4sx3TKhFigEVntiVL93.jpg,0,"[Documentary, History]",https://selectedfilms.com/marcelino-el-mejor-p...,818616,tt8223104,es,"Marcelino, el mejor payaso del mundo","An account of the life, work and many triumphs...",0.600,/btNlb2H3NXuBCN5J1FB9IVddYR.jpg,[ES],2020-07-10,0,83,[es],Released,NaN,"Marceline, the Best Clown in the World",False,6.000,2,[Estación Cinema],[ES]
309570,False,NaN,0,[],NaN,763622,tt14213656,en,Cheating Hitler: Surviving the Holocaust,"Three Canadian Holocaust survivors, with unans...",2.558,/uLfathKx9hauVKdasJc8A720kvb.jpg,[],2019-11-11,0,88,"[en, he, yi]",Released,"Three Canadian Holocaust survivors, with unans...",Cheating Hitler: Surviving the Holocaust,False,0.000,0,[],[]


In [11]:
updated_tmdb = pd.concat([tmdb_full, df]
)



updated_tmdb.reset_index(drop='index', inplace=True)

In [12]:
updated_tmdb.to_parquet("clean_datasets/updated_tmdb.parquet")

In [13]:
df = pd.read_parquet("clean_datasets/updated_tmdb.parquet")

In [14]:
df

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
0,False,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,0.0,[Comedy],None,3924.0,tt0029927,en,Blondie,Blondie and Dagwood are about to celebrate the...,2.852,/zBiHKhXklvTFwj4M1uEUcQGAVJ.jpg,[US],1938-11-30,0.0,70.0,[en],Released,The favorite comic strip of millions at last o...,Blondie,False,7.214,7.0,[Columbia Pictures],[US]
1,False,None,0.0,[Adventure],None,6124.0,tt0011436,de,Der Mann ohne Namen,None,1.091,/6xUbUCvndklbGVYiljHr34NTxSl.jpg,[DE],1921-01-01,0.0,420.0,[],Released,None,"Peter Voss, Thief of Millions",False,0.000,0.0,[],[]
2,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0.0,"[Drama, Romance]",None,8773.0,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.770,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"[DE, FR, IT, JP, PL]",1962-06-22,0.0,110.0,"[it, ja, pl, fr, de]",Released,The Intimate Secrets of Young Lovers,Love at Twenty,False,6.700,41.0,"[Ulysse Productions, Unitec Films, Cinesecolo,...","[, NZ, IT, JP, DE, PL, ]"
3,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,0.0,"[Drama, Comedy, Crime]",None,2.0,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,9.214,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,[FI],1988-10-21,0.0,73.0,[fi],Released,None,Ariel,False,7.046,248.0,[Villealfa Filmproductions],[FI]
4,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,0.0,"[Drama, Comedy, Romance]",None,3.0,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",6.282,/nj01hspawPof0mJmlgfjuLyJuRN.jpg,[FI],1986-10-17,0.0,76.0,[en],Released,None,Shadows in Paradise,False,7.182,269.0,[Villealfa Filmproductions],[FI]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316534,False,None,0.0,"[Documentaire, Musique]",None,982116.0,tt16304140,en,The Butthole Surfers Movie,None,2.794,/lhKtjx17ZcVAXEmGeuKX0sKJHMF.jpg,[],2023-05-01,0.0,0.0,[],Released,None,The Butthole Surfers Movie,False,0.000,0.0,[],[]
316535,False,None,0.0,[],None,894592.0,tt14324414,it,Upside Down,None,0.922,/r1E8DgjtaSvnJjJfYDjY4mwIVfQ.jpg,[],2023-05-01,0.0,93.0,[it],Released,None,Upside Down,False,0.000,0.0,[],[]
316536,False,/2t092J6JOS4exjhkmX01X8Vsink.jpg,0.0,[Drame],None,852865.0,tt10175106,en,Giving Birth to a Butterfly,None,4.039,/Ge2xVAtvGctwHg3tDMvSwen1zG.jpg,[US],2023-05-01,0.0,77.0,[en],Released,None,Giving Birth to a Butterfly,False,4.000,2.0,"[Dweck Productions, Hardball Entertainment]","[US, ]"
316537,False,None,0.0,[Horreur],None,772567.0,tt12165170,es,Justine,None,0.969,/6v2O5FN9gvtYFQub5XNqkIPA2Zh.jpg,[MX],2023-05-01,0.0,81.0,[es],Released,None,Justine,False,0.000,0.0,[MyIndie Productions],[AU]


In [20]:
df = df[~df["imdb_id"].duplicated(keep='last')]
df[df["original_title"].str.contains("Oppenheimer")].sort_values("imdb_id")

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
76507,False,None,0.0,[TV Movie],None,604445.0,tt0058228,de,In der Sache I. Robert Oppenheimer,None,1.349,None,[DE],1964-01-23,0.0,0.0,[de],Released,None,In der Sache I. Robert Oppenheimer,False,0.000,0.0,[],[]
313856,False,/rLb2cwF3Pazuxaj0sRXQ037tGI1.jpg,100000000.0,"[Drame, Histoire]",None,872585.0,tt15398776,en,Oppenheimer,"En 1942, convaincus que l’Allemagne nazie est ...",407.673,/boAUuJBeID7VNp4L7LNMQs8mfQS.jpg,"[GB, US]",2023-07-19,942806069.0,181.0,"[nl, en]",Released,Le monde changera à jamais.,Oppenheimer,False,8.245,4041.0,"[Syncopy, Universal Pictures, Atlas Entertainm...","[GB, US, US]"
129609,False,None,0.0,"[Documentary, History]",http://www.pbs.org/wgbh/americanexperience/fil...,401783.0,tt1559008,en,The Trials of J. Robert Oppenheimer,"J. Robert Oppenheimer was a national hero, the...",2.394,/jbIlkhgmf9PuFUSini0o91ZpyzM.jpg,[US],2008-03-24,0.0,109.0,[en],Released,None,The Trials of J. Robert Oppenheimer,False,4.400,4.0,[WGBH Boston],[US]
313888,False,/wDcf9tpfIbyQ47da81LOTNRWWwn.jpg,0.0,[Documentaire],None,1142906.0,tt27192512,en,Oppenheimer: The Real Story,None,14.397,/ecYTEmn3JBZyWbqwzjmBFfGvi3U.jpg,[],2023-07-17,0.0,97.0,[en],Released,None,Oppenheimer: The Real Story,False,7.600,5.0,[],[]
314119,False,/lv606WMZ9iZpIo6o3yeIVDAEBOl.jpg,0.0,"[Documentaire, Histoire]",None,1149947.0,tt28240284,en,To End All War: Oppenheimer & the Atomic Bomb,J. Robert Oppenheimer a dirigé l'équipe de sci...,49.204,/nHo5pZIIgAq2a9SJOUkfmldQHcB.jpg,[US],2023-07-09,0.0,88.0,[en],Released,None,"Oppenheimer, l'homme et la bombe",False,7.846,162.0,[NBC News Studios],[US]
313936,False,/t0bea3HLfxSllBdZy90UxrXl260.jpg,0.0,[Documentaire],None,1152711.0,tt28522133,en,Inside Christopher Nolan's Oppenheimer,None,22.418,/iEQQrq61v6oAtFbJ7Ym3ie5bdKA.jpg,[GB],2023-07-15,0.0,25.0,[en],Released,None,Inside Christopher Nolan's Oppenheimer,False,3.500,2.0,"[Transistor Films, Sky]","[GB, GB]"
134816,False,None,0.0,[Documentary],None,384470.0,tt5136682,es,La balada del Oppenheimer Park,The Ballad of Oppenheimer Park is a film that ...,1.126,/x2I6MgQOpOvtWGzZZ2LktFlF5MQ.jpg,[CA],2016-03-09,0.0,70.0,"[en, es]",Released,None,The Ballad of Oppenheimer Park,False,6.000,1.0,[],[]


In [ ]:
df.to_parquet("clean_datasets/updated_tmdb.parquet")

In [ ]:
with open("tmdb.json", "r") as fp:
    key = json.load(fp)

api_key = key["api_key"]

def get_movie_details(
    TMdb_id: int,
    use_IMdb: bool = False,
    IMdb_id: str = ""
):
    """Récupère les détails d'un film par son ID TMDB."""
    if use_IMdb:
        base_url = "https://api.themoviedb.org/3/find/"
        url = f"{base_url}{IMdb_id}?api_key={api_key}&external_source=imdb_id&language=fr"
    else:
        base_url = "https://api.themoviedb.org/3/movie/"
        url = f"{base_url}{movie_id}?api_key={api_key}&language=fr"
    response = requests.get(url)
    return response.json()

movie_id = 872585
movie_details = get_movie_details(TMdb_id=movie_id, use_IMdb=True, IMdb_id="tt15398776")
import pprint
pprint.pprint(movie_details)
# print(movie_details['title'])
# print(movie_details['overview'])
# print(movie_details['spoken_languages'])


In [ ]:

def get_urls() -> dict:
    return {
        "genres" : "https://api.themoviedb.org/3/genre/movie/list"
    }

def get_genres() -> dict:
    params = {
        "api_key": api_key,
        'include_adult': False,
        "language": "en",
    }

    response = requests.get(get_urls()["genres"], params)
    data = response.json()
    data
    return {
        g['id']: g["name"] for
        g in data["genres"]
    }


In [ ]:
from datetime import datetime
date = datetime.strftime(datetime.now(), '%Y-%m-%d')
base_url = "https://api.themoviedb.org/3/discover/movie"

params = {
    "api_key": api_key,
    'include_adult': False,
    "language": "en",
    "sort_by": "primary_release_date.desc",
    "primary_release_date.gte" : "2023-05-01",
    "primary_release_date.lte" : date,
    "page": 1
}
all_movies_df = pd.DataFrame()

# plutot que de faire par page, donné uniquement une date
# tmdb_id_ = []
for page in range(1, 501):
    # tmdb_id = {}
    params['page'] = page
    response = requests.get(base_url, params=params)
    data = response.json()
    page_df = pd.DataFrame(data['results'])
    all_movies_df = pd.concat([all_movies_df, page_df], ignore_index=True)
# tmdb_id_
# tmdb_id
# 1183514
all_movies_df

In [ ]:
from datetime import datetime
date = datetime.strftime(datetime.now(), '%Y-%m-%d')
base_url = "https://api.themoviedb.org/3/discover/movie"


last_day = all_movies_df.release_date.iloc[-1]

params = {
    "api_key": api_key,
    'include_adult': False,
    "language": "en",
    "sort_by": "primary_release_date.desc",
    "primary_release_date.gte" : "2023-05-01",
    "primary_release_date.lte" : last_day,
    "page": 1
}
response = requests.get(base_url, params=params)
data = response.json()
total_pages = data['total_pages']

# plutot que de faire par page, donné uniquement une date
for page in range(1, total_pages+1):
    params['page'] = page
    response = requests.get(base_url, params=params)
    data = response.json()
    page_df = pd.DataFrame(data["results"])
    all_movies_df = pd.concat([all_movies_df, page_df], ignore_index=True)


# all_movies
# 7523
all_movies_df

In [ ]:
all_movies_df.drop_duplicates(subset=["id"], keep='first', inplace=True)

In [ ]:
list_id_tmdb = all_movies_df.id.to_list()

In [ ]:
small = list_id_tmdb

In [ ]:
with open("testtt.json", "w") as fp:
    json.dump(small, fp, indent=1)

In [ ]:

with open("tmdb.json", "r") as fp:
    key = json.load(fp)

api_key = key["api_key"]

def get_movie_details(
    TMdb_id: int,
    use_IMdb: bool = False,
    IMdb_id: str = ""
):
    """Récupère les détails d'un film par son ID TMDB."""
    if use_IMdb:
        base_url = "https://api.themoviedb.org/3/find/"
        url = f"{base_url}{IMdb_id}?api_key={api_key}&external_source=imdb_id&language=fr"
    else:
        base_url = "https://api.themoviedb.org/3/movie/"
        url = f"{base_url}{TMdb_id}?api_key={api_key}&language=fr"
    response = requests.get(url)
    return [response.json()]

start = datetime.now()
movie_id = 872585
pandas = pd.DataFrame()
for id in small:
    movie_details = get_movie_details(TMdb_id=id, use_IMdb=False, IMdb_id="tt15398776")
    movies = pd.DataFrame(movie_details)
    pandas = pd.concat([pandas, movies], ignore_index=True)

end = datetime.now()
print(f"Temps écoulé : {end-start}")
# date = datetime.strftime(now, "%Y-%m-%d")
# pandas.to_csv(f"tmdb_update_{date}.csv", index=False)

In [ ]:
# # ici je recupere le dernier jour.
# last_day = all_movies_df.release_date.iloc[-1]
# # test = datetime.strptime(last_day, "%Y-%m-%d") + timedelta(days=1)
# # str_date = datetime.strftime(test,  "%Y-%m-%d")
# # str_date
# print(last_day)